<a href="https://colab.research.google.com/github/zainkhan-afk/scrabble/blob/master/wallace8x8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import os

In [0]:
transform = transforms.Compose([transforms.ToTensor(),
                              ])

transformTest = transforms.Compose([transforms.ToTensor(),
                              ])

**Loading the Training Data**

In [0]:
Data = np.loadtxt('NoisyXORTrainingData.txt')
Y = np.zeros([16, 1])
Data = Data[:, :8]

R, C = Data.shape
newData = np.zeros([2500, 16])
index = 0
for r in range(1, R, 2):
  newData[index, :8] = Data[r, :8]
  newData[index, 8:] = Data[r-1, :8] 
  index += 1
newData = np.array(newData)
print(newData.shape)

R, C = newData.shape

for d in range(0, R):
  multiplicand = 0
  multiplier = 0

  for i in range(0, int(C/2)):
    # print(d)
    multiplicand = multiplicand + newData[d, i]*2**i
    multiplier = multiplier +  newData[d, i+8]*2**i

  result = multiplicand*multiplier
  resultBin = format(int(result), "b")
  opLen = len(resultBin)
  op = []
  for i in range(0, 16):
    if i<opLen:
      op.append(int(resultBin[i]))
    else:
      op.append(0)
  op = np.array(op)
  op = op[:, np.newaxis]
  Y = np.append(Y, op, 1)

Y = np.transpose(Y)
Y = Y[1:, :]
print(Y.shape)
print(Y)

(2500, 16)
(2500, 16)
[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


**Loading the Testing Data**

In [0]:
DataTest = np.loadtxt('NoisyXORTestData.txt')
YTest = np.zeros([16, 1])
DataTest = DataTest[:, :8]
print(DataTest.shape)

R, C = DataTest.shape
newDataTest = np.zeros([2500, 16])
index = 0
for r in range(1, R, 2):
  newDataTest[index, :8] = DataTest[r, :8]
  newDataTest[index, 8:] = DataTest[r-1, :8] 
  index += 1
newDataTest = np.array(newDataTest)
print(newDataTest.shape)
R, C = newDataTest.shape

for d in range(0, R):
  multiplicand = 0
  multiplier = 0
  for i in range(0, int(C/2)):
    # print(d)
    multiplicand = multiplicand + newDataTest[d, i]*2**i
    multiplier = multiplier +  newDataTest[d, i+8]*2**i

  result = multiplicand*multiplier
  resultBin = format(int(result), "b")
  opLen = len(resultBin)
  op = []
  for i in range(0, 16):
    if i<opLen:
      op.append(int(resultBin[i]))
    else:
      op.append(0)
  op = np.array(op)
  op = op[:, np.newaxis]
  YTest = np.append(YTest, op, 1)

YTest = np.transpose(YTest)
YTest = YTest[1:, :]
print(YTest.shape)
print(YTest)

(5000, 8)
(2500, 16)
(2500, 16)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 1. 0. 0.]]


In [0]:
trainloader = torch.utils.data.DataLoader(newData, batch_size=64, shuffle=False)
labels = torch.utils.data.DataLoader(Y, batch_size=64, shuffle=False)

testloader = torch.utils.data.DataLoader(newDataTest, batch_size=64, shuffle=False)
labelsTest = torch.utils.data.DataLoader(YTest, batch_size=64, shuffle=False)

# dataiter = iter(trainloader)
# print(dataiter.shape)
# images= dataiter.next()
# print(type(images))
# print(images.shape)

# Yiter = iter(labels)
# labs = Yiter.next()
# print(labs.shape)

In [0]:
input_size = 16
hidden_sizes = [128, 64]
output_size = 16

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], output_size),
                      nn.Sigmoid())
print(model)

Sequential(
  (0): Linear(in_features=16, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=128, bias=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=128, bias=True)
  (11): ReLU()
  (12): Linear(in_features=128, out_features=128, bias=True)
  (13): ReLU()
  (14): Linear(in_features=128, out_features=128, bias=True)
  (15): ReLU()
  (16): Linear(in_features=128, out_features=16, bias=True)
  (17): Sigmoid()
)


In [0]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
time0 = time()
epochs = 1000

In [0]:
for e in range(epochs):
    running_loss = 0
    Yiter = iter(labels)
    for images in trainloader:
        # Flatten MNIST images into a 784 long vector
        # images = images.view(images.shape[0], -1)
        labs = Yiter.next()
        # Training pass
        optimizer.zero_grad()
        
        # print("IP: ", images.shape)
        # print(images.shape)
        output = model(images.float())
        # print("OP: ", output.shape)
        # print("Truth: ", labs.shape)
        loss = criterion(output, labs.float())
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    correct_count, all_count = 0, 0
    Yiter = iter(labelsTest)
    for images in testloader:
        output = model(images.float())
        output = output.round() 
        # print(output.round())
        labs = Yiter.next()
        # print("OP: ", output.shape)
        # print("TRUTH: ", labs.shape)
        R, C = output.shape
        for r in range(0, R):
          # print(output[r, :].shape, labs[r,:].shape)
          res = abs(output[r, :] - labs[r, :])
          # print(res.shape)
          res = sum(res)
          if res == 0:
            correct_count += 1
          all_count += 1
        # for i in range(len(labels)):
        #   img = images[i].view(1, 784)
        #   with torch.no_grad():
        #       logps = model(img)

          
        #   ps = torch.exp(logps)
        #   probab = list(ps.numpy()[0])
        #   pred_label = probab.index(max(probab))
        #   true_label = labels.numpy()[i]
        #   if(true_label == pred_label):
        #     correct_count += 1
        #   all_count += 1

    print("Number Of Samples Tested =", all_count)
    print("\nModel Accuracy =", (correct_count/all_count))
    # else:
    print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 0 - Training loss: 0.2182927131652832
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 1 - Training loss: 0.20259459801018237
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 2 - Training loss: 0.20244840830564498
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 3 - Training loss: 0.20239742584526538
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 4 - Training loss: 0.20235022976994516
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 5 - Training loss: 0.20231434106826782
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 6 - Training loss: 0.2022845223546028
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 7 - Training loss: 0.20225820913910866
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 8 - Training loss: 0.20223506204783917
Number Of Samples Tested = 2500

Model Accuracy = 0.0016
Epoch 9 - Training loss: 0.

In [0]:
correct_count, all_count = 0, 0
Yiter = iter(labelsTest)
for images in testloader:
  output = model(images.float())
  output = output.round() 
  # print(output.round())
  labs = Yiter.next()
  # print("OP: ", output.shape)
  # print("TRUTH: ", labs.shape)
  R, C = output.shape
  for r in range(0, R):
    # print(output[r, :].shape, labs[r,:].shape)
    res = abs(output[r, :] - labs[r, :])
    # print(res.shape)
    res = sum(res)
    if res == 0:
      correct_count += 1
    all_count += 1
  # for i in range(len(labels)):
  #   img = images[i].view(1, 784)
  #   with torch.no_grad():
  #       logps = model(img)

    
  #   ps = torch.exp(logps)
  #   probab = list(ps.numpy()[0])
  #   pred_label = probab.index(max(probab))
  #   true_label = labels.numpy()[i]
  #   if(true_label == pred_label):
  #     correct_count += 1
  #   all_count += 1

print("Number Of Samples Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Samples Tested = 2500

Model Accuracy = 0.0008
